<a href="https://colab.research.google.com/github/PauBea/spark/blob/main/delta_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%rm -rf spark-warehouse
!pip install -q pyspark
!pip install -q delta-spark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from delta import configure_spark_with_delta_pip
from delta import DeltaTable

builder = (
    SparkSession.builder.appName ("Delta Tutorial")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
def _enable_sparkui(port=4040):
  from google.colab import output
  return output.serve_kernel_port_as_window(port, path= '/jobs/index.html')

  _enable_sparkui()

In [ ]:
df = spark.read.options(inferSchema =True, header= True).csv("/content/sample_data/california_housing_train.csv")
df.printSchema()
df.show(5)



root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|     

In [ ]:
spark.sql("CREATE DATABASE delta_tutorial")
spark.sql("USE delta_tutorial")

DataFrame[]

In [ ]:
df.write.mode("overwrite").format("delta").saveAsTable("california_housing")

In [ ]:
california_housing_df = DeltaTable.forName(spark, "delta_tutorial.california_housing")

In [ ]:
spark.sql(
    """
    SELECT
        *
    FROM
      delta_tutorial.california_housing

    """
).show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

In [ ]:
spark.sql("""
  INSERT INTO delta_tutorial.california_housing
  VALUES(1,1,10,5000,1000,1000,300,1.5,70000)

""")

DataFrame[]

In [ ]:
spark.sql("""
  SELECT
    *
  FROM
    delta_tutorial.california_housing
  WHERE
  longitude = 1


""").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|      1.0|     1.0|              10.0|     5000.0|        1000.0|    1000.0|     300.0|          1.5|           70000.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+

